In [1]:
import caffe
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import os
%matplotlib inline

In [2]:
net = caffe.Net('xiaojing_dilated_convolution/no_pad_dia/11/deploy_denoise.prototxt',
                'xiaojing_dilated_convolution/no_pad_dia/new_set/11/denoising_net_iter_1500000.caffemodel', 
                caffe.TEST )

In [3]:
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
#transformer.set_mean('data',104)

In [4]:
#net.blobs['data'].reshape(1,1,400,275)

In [5]:
val_images_path = '/home/rgsl888/Finger_Print_dataset/validation_input_back/'
val_out_path = '/home/rgsl888/Finger_Print_dataset/validation_out_xiaojing_dia_no_pad_1500000/'
img_ext = '.jpg'

if not os.path.exists(val_images_path):
    os.makedirs(val_images_path)
if not os.path.exists(val_out_path):
    os.makedirs(val_out_path)
images = glob.glob (val_images_path + "/*" + img_ext)


In [6]:
for img in images:
    im = cv2.imread(img,0)
    im = im - np.array((104.00698793)) 
    net.blobs['data'].data[...] = transformer.preprocess('data', im)
    out = net.forward()
    #plt.imshow(out['output'][0][0])
    img_name = img.split('/')
    denoised_img = out['output'][0][0]
    denoised_img += np.array((104.00698793))
    denoised_img = np.clip(np.rint(denoised_img), 0, 255.0)
    #denoised_img = cv2.resize(denoised_img, (275, 400))
    cv2.imwrite(val_out_path + img_name[-1], denoised_img)